In [1]:
def poisson_pmf(y, x, w):
    inv_link_fn = np.exp(x*w)
    
    p = inv_link_fn**y * np.exp(-inv_link_fn) / math.factorial(y)
    
    return p

In [ ]:
def bernoulli_pmf(y, x, w):
    inv_link_fn = sigmoid(x, w)
    
    p = inv_link_fn**y * (1 - inv_link_fn)**(1 - y)
    
    return p